Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.4 MB/s eta 0:00:00


Cirq

In [2]:
import cirq
import numpy as np

# Qubitler
alice = cirq.LineQubit(0)
bob = cirq.LineQubit(1)

# Devre oluştur
circuit = cirq.Circuit()

# 1. Bell durumu oluştur
circuit.append(cirq.H(alice))
circuit.append(cirq.CNOT(alice, bob))

# 2. Alice bitleri kodlasın (ör: 10 -> Z kapısı)
circuit.append(cirq.Z(alice))  # 10 bit için

# 3. Bob Bell ölçümü
circuit.append(cirq.CNOT(alice, bob))
circuit.append(cirq.H(alice))
circuit.append([cirq.measure(alice, key='a'), cirq.measure(bob, key='b')])

# Simüle et
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1)
print("Gönderilen bit (Alice -> Bob):", result)


Gönderilen bit (Alice -> Bob): a=1
b=0


Qiskit

In [4]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

# 2 qubit devre
qc = QuantumCircuit(2,2)

# Bell durumu
qc.h(0)
qc.cx(0,1)

# Alice bitleri kodlasın (ör: 01 -> X kapısı)
qc.x(0)

# Bob ölçümü için Bell bazına dönüştür
qc.cx(0,1)
qc.h(0)

# Ölçüm
qc.measure([0,1],[0,1])

# Simülasyon
backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1).result()
counts = result.get_counts()
print("Gönderilen bit (Alice -> Bob):", counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP

In [5]:
from qutip import basis, tensor, qeye, sigmax, sigmay, sigmaz, ket2dm

# Başlangıç Bell durumu
psi0 = (tensor(basis(2,0), basis(2,0)) + tensor(basis(2,1), basis(2,1))).unit()
rho = ket2dm(psi0)

# Alice 2-bit kodlama (ör: 10 -> Z)
rho = (sigmaz() * rho.ptrace(0)).expand(tensor(qeye(2), qeye(2)))

# Bob ölçümü için Bell dönüşümü (H ve CNOT)
# H Alice, CNOT Alice->Bob
from qutip import hadamard_transform, cnot
U = cnot() * tensor(hadamard_transform(), qeye(2))
rho_final = U * rho * U.dag()

print(rho_final)


AttributeError: 'Qobj' object has no attribute 'expand'